In [13]:
from langgraph.graph import StateGraph, START, END
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.tools import tool
from langchain_community.tools import ShellTool
from langgraph.prebuilt import ToolNode
from typing import TypedDict, List
import os
from dotenv import load_dotenv
from github import Github
load_dotenv()

True

In [14]:
# Set up Groq LLM
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")  

llm = ChatGroq(
    model="openai/gpt-oss-20b",
)

In [15]:
# Define the state
class CodeReviewState(TypedDict):
    repo_name: str
    pr_number: int
    code_changes: List[str]
    feedback: List[str]
    issues: List[str]
    suggestions: List[str]

In [10]:
# Define nodes
def analyze_code(state: CodeReviewState):
    code = state["code"]
    prompt = f"""
    Analyze the following code for potential issues, bugs, and improvements.
    Code:
    {code}
    
    Provide a list of issues and suggestions.
    """
    response = llm.invoke([SystemMessage(content="You are a code review expert."), HumanMessage(content=prompt)])
    analysis = response.content
    
    # Parse analysis into issues and suggestions (simple split)
    lines = analysis.split('\n')
    issues = [line for line in lines if 'issue' in line.lower() or 'bug' in line.lower()]
    suggestions = [line for line in lines if 'suggest' in line.lower() or 'improve' in line.lower()]
    
    return {"issues": issues, "suggestions": suggestions}

def provide_feedback(state: CodeReviewState):
    issues = state["issues"]
    suggestions = state["suggestions"]
    feedback = f"Issues: {issues}\nSuggestions: {suggestions}"
    return {"feedback": [feedback]}

In [ ]:
# Fetch PR changes
def fetch_pr_changes(state: CodeReviewState):
    repo_name = state["repo_name"]
    pr_number = state["pr_number"]
    
    g = Github(os.getenv("GITHUB_TOKEN"))
    repo = g.get_repo(repo_name)
    pr = repo.get_pull(pr_number)
    
    files = pr.get_files()
    code_changes = []
    for file in files:
        if file.patch:
            code_changes.append(f"File: {file.filename}\n{file.patch}")
    
    return {"code_changes": code_changes}

# Define nodes
def analyze_code(state: CodeReviewState):
    code_changes = state["code_changes"]
    all_code = "\n\n".join(code_changes)
    prompt = f"""
You are an expert code reviewer. Analyze the following code changes from a GitHub pull request for potential issues, bugs, security vulnerabilities, performance problems, and improvements. Provide a thorough, structured review.

Code Changes:
{all_code}

Please structure your response with the following sections:

1. **Security Issues**: Identify any security vulnerabilities (e.g., SQL injection, XSS, authentication flaws, data exposure).

2. **Bugs and Logic Errors**: Point out any bugs, logical errors, or incorrect implementations.

3. **Performance Concerns**: Highlight any potential performance issues or inefficiencies.

4. **Code Quality and Best Practices**: Suggest improvements for readability, maintainability, adherence to coding standards, and best practices.

5. **Testing and Coverage**: Recommend additional tests or improvements to existing tests.

6. **Documentation**: Note if documentation needs to be updated or added.

7. **Other Issues**: Any other concerns not covered above.

For each issue or suggestion, provide:
- A brief description
- Severity level (Critical, High, Medium, Low)
- Specific location in the code (file and line if possible)
- A suggested fix or improvement

Be specific, actionable, and constructive. If no issues are found in a category, state "No issues found."
"""
    response = llm.invoke([SystemMessage(content="You are a code review expert."), HumanMessage(content=prompt)])
    analysis = response.content
    
    # Parse analysis into issues and suggestions (simple split)
    lines = analysis.split('\n')
    issues = [line for line in lines if 'issue' in line.lower() or 'bug' in line.lower() or 'vulnerability' in line.lower() or 'error' in line.lower()]
    suggestions = [line for line in lines if 'suggest' in line.lower() or 'improve' in line.lower() or 'recommend' in line.lower()]
    
    return {"issues": issues, "suggestions": suggestions}

def provide_feedback(state: CodeReviewState):
    issues = state["issues"]
    suggestions = state["suggestions"]
    feedback = f"**Code Review Feedback**\n\n**Issues:**\n" + "\n".join(f"- {issue}" for issue in issues) + f"\n\n**Suggestions:**\n" + "\n".join(f"- {sugg}" for sugg in suggestions)
    return {"feedback": [feedback]}

def post_feedback(state: CodeReviewState):
    repo_name = state["repo_name"]
    pr_number = state["pr_number"]
    feedback = state["feedback"][0] if state["feedback"] else "No feedback generated."
    
    g = Github(os.getenv("GITHUB_TOKEN"))
    repo = g.get_repo(repo_name)
    pr = repo.get_pull(pr_number)
    
    pr.create_issue_comment(feedback)
    return {}

In [ ]:
# Build the graph
graph = StateGraph(CodeReviewState)

graph.add_node("fetch_pr", fetch_pr_changes)
graph.add_node("analyze", analyze_code)
graph.add_node("feedback", provide_feedback)
graph.add_node("post", post_feedback)

graph.add_edge(START, "fetch_pr")
graph.add_edge("fetch_pr", "analyze")
graph.add_edge("analyze", "feedback")
graph.add_edge("feedback", "post")
graph.add_edge("post", END)

code_review_agent = graph.compile()

In [23]:
# Test the agent with PR
# Example: repo_name = "owner/repo", pr_number = 1
result = code_review_agent.invoke({
    "repo_name": "shrivarshapoojari/SmartReview",  # Replace with actual repo
    "pr_number": 1,  # Replace with actual PR number
    "code_changes": [],
    "feedback": [],
    "issues": [],
    "suggestions": []
})

# Display the result in a readable way
print("=== Code Review Agent Result ===")
print(f"Repository: {result.get('repo_name', 'N/A')}")
print(f"PR Number: {result.get('pr_number', 'N/A')}")
print("\n--- Code Changes ---")
for i, change in enumerate(result.get('code_changes', []), 1):
    print(f"{i}. {change[:100]}...")  # 
print("\n--- Issues ---")
for issue in result.get('issues', []):
    print(f"- {issue}")
print("\n--- Suggestions ---")
for suggestion in result.get('suggestions', []):
    print(f"- {suggestion}")
print("\n--- Feedback ---")
for feedback in result.get('feedback', []):
    print(feedback)

C:\Users\shree\AppData\Local\Temp\ipykernel_30528\957474520.py:6: DeprecationWarning: Argument login_or_token is deprecated, please use auth=github.Auth.Token(...) instead
  g = Github(os.getenv("GITHUB_TOKEN"))
C:\Users\shree\AppData\Local\Temp\ipykernel_30528\957474520.py:50: DeprecationWarning: Argument login_or_token is deprecated, please use auth=github.Auth.Token(...) instead
  g = Github(os.getenv("GITHUB_TOKEN"))
C:\Users\shree\AppData\Local\Temp\ipykernel_30528\957474520.py:50: DeprecationWarning: Argument login_or_token is deprecated, please use auth=github.Auth.Token(...) instead
  g = Github(os.getenv("GITHUB_TOKEN"))


=== Code Review Agent Result ===
Repository: shrivarshapoojari/SmartReview
PR Number: 1

--- Code Changes ---
1. File: research/research.ipynb
@@ -2,7 +2,7 @@
  "cells": [
   {
    "cell_type": "code",
-   "execut...
2. File: test.py
@@ -0,0 +1,53 @@
+from flask import Flask, request, render_template_string
+import sql...

--- Issues ---
- Below is a systematic review of the code, pointing out bugs, security problems, and practical improvements.
- ### 2.1 Functional / Runtime Issues
- | Issue | Description | Fix / Improvement |
- | **`debug=True` in production** | Exposes stack traces and auto‑reloader. | Use `debug=False` or `app.run()` without the flag in production. |
- ### 2.2 Security Issues
- | Issue | Description | Fix / Mitigation |
- | **Debug Mode** | Exposes stack traces. | Disable in production. |
- | Issue | Description | Fix / Improvement |
-     app.run(debug=False)
- - **`debug=False`** for production.  
- 8. **Disable debug mode** in production.  
- - **`test.py`**: A 